# **IPP producer reservoir operation model**

In [ ]:
!pip install pyomo
!apt-get install -y -qq glpk-utils
!apt-get install -y coinor-cbc

In [ ]:
from pyomo.environ import *
#Import solver
opt=SolverFactory('cbc')

#opt = SolverFactory('appsi_highs')  # use the HiGHS solver via APPSI

## Creating Model with vectors and numpy

In [ ]:
#Importing numpy library to enable array use and pandas for data frames
import numpy as np
import pandas as pd

#Entering problem data
inflow = np.array([2, 2, 3, 4, 3, 2, 2, 1, 2, 3, 3, 2]) #UoW inflow
elect_price = np.array([1.6, 1.7, 1.8, 1.9, 2.0, 2.0, 2.0, 1.9, 1.8, 1.7, 1.6, 1.5]) #electricity price
irrig_price = np.array([1.0, 1.2, 1.8, 2.0, 2.2, 2.2, 2.5, 2.2, 1.8, 1.4, 1.1, 1.0]) #irrigation price
r0 = 5 #reservoir initial level
r12 = 5 #reservoir level in last month

T=range(12)  #twelve months of the year

In [ ]:
#Creating model
model = ConcreteModel()

#Index decision variables
model.e=Var(T,domain=NonNegativeReals) #dec var for electricity generation i.e. how much water for electricity generation == water released
model.i=Var(T,domain=NonNegativeReals) #dec var for irrigation i.e. how much water for irrigation
model.r=Var(T,domain=NonNegativeReals) #dec var tracking reservoir level from one month to the other

#Adding objective function
model.profit = Objective(
    expr = sum(elect_price[t]*model.e[t] for t in T) + sum(irrig_price[t]*model.i[t] for t in T),
    sense= maximize
    )

In [ ]:
#Adding constraints
model.constraints = ConstraintList()
for t in T:
    #Need at least 1 unit of water downstream
    model.constraints.add(1 <= model.e[t] - model.i[t])

    #reservoir water balance constraint
    if (t==0):
      model.constraints.add(model.r[t] == r0 + inflow[t] - model.e[t])
    else:
      model.constraints.add(model.r[t-1] + inflow[t] - model.e[t] == model.r[t])

    #reservoir level in the last month should be 5
    if (t==11):
      model.constraints.add(model.r[t] == r12)

    #simple bound e[t]: total outflow cannot exceed 7 units of water
    model.constraints.add(model.e[t] <= 7)

    #simple bound r[t]: reservoir level at any month cannot exceed 10 unit of water
    #model.constraints.add(0 <= model.r[t])
    model.constraints.add(model.r[t] <= 10)

In [ ]:
#Solving model
#opt.solve(model)

results = opt.solve(model, tee=True)

#print(results.solver.termination_condition)

#Print results
print('Profit = ', model.profit())
#print('\nDecision Variables')


In [ ]:
sys_op_sch = pd.DataFrame(np.zeros((12,4)))

for t in T:
  sys_op_sch.iloc[t,0] = t+1
  sys_op_sch.iloc[t,1] =  model.e[t].value
  sys_op_sch.iloc[t,2] =  model.i[t].value
  sys_op_sch.iloc[t,3] =  model.r[t].value

sys_op_sch = sys_op_sch.rename(columns={0:"Month",1:"electricity generation",2:"irrigation",3:"reservoir level"})
print(sys_op_sch)

    Month  electricity generation  irrigation  reservoir level
0     1.0                     1.0         0.0              6.0
1     2.0                     1.0         0.0              7.0
2     3.0                     1.0         0.0              9.0
3     4.0                     3.0         2.0             10.0
4     5.0                     3.0         2.0             10.0
5     6.0                     7.0         6.0              5.0
6     7.0                     7.0         6.0              0.0
7     8.0                     1.0         0.0              0.0
8     9.0                     2.0         1.0              0.0
9    10.0                     1.0         0.0              2.0
10   11.0                     1.0         0.0              4.0
11   12.0                     1.0         0.0              5.0


In [ ]:
model.dual=Suffix(direction=Suffix.IMPORT)

In [ ]:
#Solve Model
results = opt.solve(model)

print("Shadow Prices")
model.dual.pprint()

Shadow Prices
dual : Direction=IMPORT, Datatype=FLOAT
    Key             : Value
    constraints[10] :  -3.9
    constraints[11] :  -0.0
    constraints[12] :  -0.0
    constraints[13] :  -2.0
    constraints[14] :  -3.9
    constraints[15] :  -0.0
    constraints[16] :   0.3
    constraints[17] :  -2.2
    constraints[18] :  -4.2
    constraints[19] :  -0.0
     constraints[1] :  -2.3
    constraints[20] :  -0.0
    constraints[21] :  -2.2
    constraints[22] :  -4.2
    constraints[23] :  -0.0
    constraints[24] :  -0.0
    constraints[25] :  -2.5
    constraints[26] :  -4.2
    constraints[27] :   0.3
    constraints[28] :  -0.0
    constraints[29] :  -2.2
     constraints[2] :   3.9
    constraints[30] :  -4.1
    constraints[31] :  -0.0
    constraints[32] :  -0.0
    constraints[33] :  -1.8
    constraints[34] :  -3.6
    constraints[35] :  -0.0
    constraints[36] :  -0.0
    constraints[37] :  -1.9
    constraints[38] :  -3.6
    constraints[39] :  -0.0
     constraints[3] : 